# Neurons as logic gates

## A little excursus on booleans and logic gates

Data with only two possible values (think of a neuron that can only produce output which is either 1 or 0) is handled by a branch of mathematics called _Boolean algebra_, where the values 1 and 0 are often replaced by the labels __true__ and __false__. Boolean algebra comes with a set of basic operations or functions, which take as input boolean values and produce boolean output. An example is the OR function, which given two boolean inputs, returns `True` if at least one of the inputs is `True`, or the exclusive or (XOR) function, which returns `True` only if exactly one of the two inputs is true. These operations are often summarized by so-called __truth tables__, like the one below (where $x$ and $y$ are the two inputs to the function).


| x | y | x AND y | x OR y | x XOR y |
|---|---|---------|--------|---------|
| 0 | 0 |0|0|0
| 1 | 0 |0|1|1
| 0 | 1 |0|1|1
| 1 | 1 |1|1|0

Most programming languages implement a boolean data type, which can be represented by 0 and 1 or by `True` and `False` labels. In Python, `True` and `False` are built-in constants, and we can ask Python to combine them with Boolean operations.

In [ ]:
a = True
b = False
print(a or b)

We can also use the numeric values 1 and 0, as long as we tell Python that we mean boolean and not integer values. 

In [ ]:
a = bool(1)
b = bool(0)
print(a or b)

In electronics, a __logic gate__ is a (conceptual) device which implements a boolean function. We can think of neurons as _biological logic gates_, in the sense that we can consider our modeled neuron as a system which receives binary inputs (e.g. spike or no spike from its presynpatic neurons) and outputs a binary signal (spike or no spike). In this tutorial we will explore how, depending on how the inputs are combined, a neuron can implement different boolean functions.

## Part 1 - A single neuron

We begin by writing a very simply Python class which we call `neuron`. This neuron has two methods, namely `__init__`, which is used to generate instances of this class, and `output`, which produces the neuron's output given the inputs. Below is also an example of how you can instantiate a neuron with all its parameters and call the `output` method (you don't _need_ to understand the code, but do take a look if you are interested).

In [ ]:
import numpy as np
from utils import ReLu, sigmoid, hardlim, hardlims, purelin, tanh
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

class neuron():
    def __init__(self, bias, transfer_function, weights):
        self.bias = bias
        self.transfer_function = transfer_function
        self.weights = weights

    def output(self, inputs):
        # compute a weighted sum of the inputs
        x = np.sum([w * p for w, p in zip(inputs, self.weights)])
        # pass the sum through the activation function to produce the output
        return self.transfer_function(x - self.bias)


def neuron_for_widget(bias, transfer_function, weight_1, weight_2, input_1, input_2):
    my_neuron = neuron(bias=bias, transfer_function=transfer_function, weights=[weight_1, weight_2])
    out = my_neuron.output(inputs=[input_1, input_2])
    return bool(out)

# you can create an instance of the class neuron with a certain bias,
# transfer function, and with a list of weights that connect it to its input neurons
my_neuron = neuron(bias=0, transfer_function=hardlims, weights=[1, -1])

# then you can call the output method with a list of inputs to get the output
# of the neuron
out = my_neuron.output(inputs=[1, -1])
print('neuron output = ', out)

### Question 1
Write down the general equation that describes output $a$ as function of the inputs $p_i$ for some transfer function $f$. Then, simplify the equation for the case of a linear transfer function and two inputs $p_1$ and $p_2$.

### Answer 1
General equation: <br>
Simplified:

### Question 2
To get us started we consider a neuron with a `hardlim` transfer function. Run the cell below, then use the sliders to set the `bias=1`, `weight_1=1`, `weight_2=-1`. Then, try all possible four combinations of input values, and fill out a truth table like the one above, corresponding to the boolean operation performed by this neuron. Note that in the answer cell I already created the table for you, and listed the combinations of values of $p_1$ and $p_2$: you just have to edit the last column.

In [ ]:
interact(neuron_for_widget, 
         bias=(0, 1, 0.5), transfer_function=fixed(hardlim), 
         weight_1=(-1, 1, 0.5), weight_2=(-1, 1, 0.5), input_1=(0, 1, 1), input_2=(0, 1, 1));

### Answer 2
| $p_1$ | $p_2$ | neuron |
|---|---|--------|
| 0 | 0 | | 
| 1 | 0 | |
| 0 | 1 | |
| 1 | 1 | |

### Question 3
Next, run the cell below, and find values of the bias and the weights such that the output of our neuron becomes equivalent to the AND function shown previously.

In [ ]:
interact(neuron_for_widget, 
         bias=(0, 1, 0.5), transfer_function=fixed(hardlim), 
         weight_1=(-1, 1, 0.5), weight_2=(-1, 1, 0.5), input_1=(0, 1, 1), input_2=(0, 1, 1));

### Answer 3
[your answer]

### Question 4
Now find parameter values such that the computation performed by the neuron corresponds to the OR function.

In [ ]:
interact(neuron_for_widget, 
         bias=(0, 1, 0.5), transfer_function=fixed(hardlim), 
         weight_1=(-1, 1, 0.5), weight_2=(-1, 1, 0.5), input_1=(0, 1, 1), input_2=(0, 1, 1));

### Answer 4
[your answer]

### Question 5
Can you also find parameter values which allow our neuron to compute the XOR operation?

### Answer 5
[your answer]

### Question 6
Can you show mathematically why this is not possible? Hint: write out the equations for all combinations of input values (and fill in 0 for the bias).

### Answer 6
[your answer]

Another way to think about what the neuron is doing is to plot each combination of input values on a 2-D plane. Then, the parameters of our neuron determine a line on this 2-D plane that will separate points in input space for which the neuron becomes active (responds with a 1) from points where the neuron remains inactive (responds with a 0). This line is known as the _decision boundary_. 

### Question 7
Derive the equation for the line which represents the decision boundary, through expressing p_2 as function of p_1. (this will allow you to visualize the boundary)

### Answer 7
[your answer]

Let's write up a function which will allow us to visualize the decision boundary.

In [ ]:
import matplotlib.pyplot as plt

def plot_decision_boundary(weight_1, weight_2, bias):
    inputs = np.array([[0, 0],
                       [1, 1],
                       [0, 1],
                       [1, 0]])
    
    weights = [weight_1, weight_2]
    my_neuron = neuron(bias=bias, transfer_function=hardlim, weights=weights)

    f, ax = plt.subplots(1, 1, figsize=[6, 6])
    
    plot_line = lambda x : (bias-weights[0]*x)/ (weights[1] + 10e-5)
    xx, yy = np.arange(-2, 2, step=0.001), np.arange(-2, 2, step=0.001)

    ax.plot(xx, plot_line(xx), c='k')
    for i in range(inputs.shape[0]):
        if my_neuron.output(inputs[i,:])>0:
            ax.scatter(inputs[i, 0], inputs[i, 1], facecolors='k', edgecolors='k', s=104, linewidths=2)
        else:
            ax.scatter(inputs[i, 0], inputs[i, 1], facecolors='none', edgecolors='k', s=104, linewidths=2)
    ax.set_xlim([-1, 2])
    ax.set_ylim([-1, 2])
    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_title('activation function: hardlim')
    plt.show()

Run the cell below, and using the sliders, explore the effect of the weights and of the bias on the decision boundary. Filled dots indicate that the neuron's output is 1 (true), and 0 (false) for the empty dots. 

In [ ]:
interact(plot_decision_boundary, weight_1=(-1, 1, 0.1), weight_2=(-1, 1, 0.1), bias=(-1, 1, 0.05));

### Question 8
Set up the weights and bias you discovered in question 4 which implement the OR function. Do you get the expected filled/empty dots? 

### Answer 8
[your answer]

### Question 9
What would the decision boundary have to look like for the XOR operation to be carried out? Give a graphical explanation of why XOR cannot be solved by a single linear classifier. 

### Answer 9
[your answer]

## Part 2 - Extending to more layers

For now we have focused our attentions on mechanisms which pertain to the functioning of a single neuron. But there must be a reason why the brain contains so many neurons! Intuitively, having more neurons should allow us to somehow do more complicated things. Although it is certainly not the only factor, the number of neurons tends to be larger in more evolved and cognitively capable species, as you can see [here](https://en.wikipedia.org/wiki/List_of_animals_by_number_of_neurons)). In this part of the assignment we are going to show in a very simple yet very concrete example how using more neurons allows us to do more sophisticated computations. 

If you have completed Part 1 of the assignment it should be clear that the XOR problem (associating a response equal to 1 only when either one of the inputs is 1, and 0 otherwise) is just too much for a single neuron. At this point, you may understandably be disappointed in neurons, and wonder how the brain ever does anything computationally interesting. But don't lose faith: __the strength is in the union of many computationally simple elements__.

### Question 1
Consider again the decision boundary plot. Suppose now you can draw two straight lines instead of one. Can you now draw them in such a way as to compute the XOR function?

### Answer 1
[your answer]

It seems like with two neurons we could be getting somewhere! But how do we proceed? The idea is that we can give the same input to two neurons, which can then process it independently (with different weights and biases) and give us back two outputs. Then we can have a neuron which takes as input the output of the first two neurons and gives us the final output. So, while the first and second neuron (the first layer) compute a logic function on the input, the neuron in the second layer (neuron 3) computes a logic function on the output of the first layer. This is summarized in the figure below: on the left we have the one neuron scenario, and on the right the two-layer architecture.

![title](images/lg_1.svg)

### Question 2
In question 1 you were hopefully able to draw two lines that isolate (in the intersection of their half-planes) the points at $(1, 0)$ and $(0, 1)$. Translate this into truth table entries by filling in the __Output neuron 1__ and __Output neuron 2__ below.

### Answer 2
[Edit the entries in the table for neuron 1 and neuron 2]

| Input 1 | Input 2 | Output neuron 1 | Output neuron 1 | Output neuron 3 |
|---------|---------|-----------------|-----------------|-----------------|
|    0    |    0    |      ?          |        ?        |        0        | 
|    0    |    1    |      ?          |        ?        |        1        |
|    1    |    0    |      ?          |        ?        |        1        |
|    1    |    1    |      ?          |        ?        |        0        |

### Question 3
What logic function allows you to combine the outputs of neuron 1 and neuron 2 to obtain the desired XOR at the output of neuron 3?

### Answer 3
[your answer]

### Question 4

We are now going to implement this network using the `neuron` object introduced at the beginning of this assignment. In the code below, we generate three neurons, and use them to compute the chained logic operation which allows us to compute the XOR function on our inputs. The output of network is computed for all input combinations. We then print out a truth table where the three columns correspond to input 1, input 2, and the final output (the output of the third neuron).

Find values for the weights and biases of the three neurons in our small network such that the output of the third neuron gives the correct XOR function on the inputs. Edit the code below by inserting the correct parameters, and run it to make sure that you get the desired output. You can use the decision boundary interactive plot from before to determine the parameters of each neuron. 

At the beginning, everything is initialized to 0: if you run the cell once you will see that you get a truth table printed out with only zeros in the output).

In [ ]:
# we construct our three neurons
neuron1 = neuron(bias=0, transfer_function=hardlim, weights=[0, 0])
neuron2 = neuron(bias=0, transfer_function=hardlim, weights=[0, 0])
neuron3 = neuron(bias=0, transfer_function=hardlim, weights=[1, 1])

# we make an array with all possible input combinations
inputs = np.array([[0, 0],
                   [0, 1],
                   [1, 0],
                   [1, 1]])

# for every input, we feed it to neuron 1 and neuron 2, and record their output.
# then, we feed the output of the first layer (neurons 1 and 2) to the third neuron,
# to get out final output.
for inpt in inputs:
    out1 = neuron1.output(inpt)
    out2 = neuron2.output(inpt)
    out3 = neuron3.output([out1, out2]) 
    print("%s | %s | %s " %(inpt[0], inpt[1], out3)) # print the results. the last column is the output of neuron 3

### Answer 4
[Edit the code in the cell above]

### Question 5

It really does seem the strength is in the union of many computationally simple elements. Even to the extent that there is not just one solution to this problem but two. Given that you have found one solution by which the output of two intermediate neurons can be combined in a third neuron to solve the XOR problem, what is the second solution?

Go over the previous questions and answer for this other scenario:

What are truth table entries for __Output neuron 1__ and __Output neuron 2__ and __Output neuron 3__?

What logic function allows you to combine the outputs of neuron 1 and neuron 2 to obtain the desired XOR at the output of neuron 3?

Find values for the weights and biases of the three neurons in our small network such that the output of the third neuron gives the correct XOR function on the inputs. Edit the code below by inserting the correct parameters, and run it to make sure that you get the desired output. (Again: You can use the decision boundary interactive plot from before to determine the parameters of each neuron. )


### Question 5a
[Edit the entries in the table for neuron 1 and neuron 2]

| Input 1 | Input 2 | Output neuron 1 | Output neuron 1 | Output neuron 3 |
|---------|---------|-----------------|-----------------|-----------------|
|    0    |    0    |      ?          |        ?        |        0        | 
|    0    |    1    |      ?          |        ?        |        1        |
|    1    |    0    |      ?          |        ?        |        1        |
|    1    |    1    |      ?          |        ?        |        0        |

### Question 5b
What logic function allows you to combine the outputs of neuron 1 and neuron 2 to obtain the desired XOR at the output of neuron 3?

### Question 5c
Find values for the weights and biases of the three neurons for the second solution:

In [ ]:
# we construct our three neurons
neuron1 = neuron(bias=0, transfer_function=hardlim, weights=[0, 0])
neuron2 = neuron(bias=0, transfer_function=hardlim, weights=[0, 0])
neuron3 = neuron(bias=0, transfer_function=hardlim, weights=[1, 1])

# we make an array with all possible input combinations
inputs = np.array([[0, 0],
                   [0, 1],
                   [1, 0],
                   [1, 1]])

# for every input, we feed it to neuron 1 and neuron 2, and record their output.
# then, we feed the output of the first layer (neurons 1 and 2) to the third neuron,
# to get out final output.
for inpt in inputs:
    out1 = neuron1.output(inpt)
    out2 = neuron2.output(inpt)
    out3 = neuron3.output([out1, out2]) 
    print("%s | %s | %s " %(inpt[0], inpt[1], out3)) # print the results. the last column is the output of neuron 3